### Instructions: Follow the instructions provided in each step, or in the output from a cell 
Step 1
* Make sure the python notebook and the pivot table Excel file are in the **same** folder on your computer.
* Enter the name of the pivot table in the next cell, then press the 'run' button above.
* The python notebook will give you a preview of what will be used in the analysis beneath the cell after you press the run button.  
  * Scan the row names to verify that they are the same as in the Excel sheet that you want ot use.  If they are, skip the following cells and proceed to Step 2.
* Otherwise, follow the instructions in the output.

In [1]:
pivot_table = '20411_Felber1-2_MHC-I_Haplotypes_23Mar18.xlsx' # enter name here

In [4]:
import pandas as pd
from pandas import DataFrame
import re
import time
import math

def parseExcelWithPandas(fName, excelFileP):
    eSheetData = ''
    dfs = {sheet_name: excelFileP.parse(sheet_name)
      for sheet_name in excelFileP.sheet_names}
    if not fName:
        sheetCount = -1
        for s in dfs:
            sheetCount += 1
            m = re.search('pivot', s)
            m2 = re.search('MiSeq', s)
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            elif m2:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    else:
        for s in dfs:
            m = re.search(fName, s) # case insensitive?
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    return eSheetData

def openFileAsList(f):
    l = []
    with open(f, 'r') as fOpen:
        for i in fOpen:
            i = i.rstrip('\r\n')
            l.append(i)
    return l

def findColumnIdxStartStop(pdDF):
    xCt = -1
    xStart = -1
    xStop = -1
    foundInitialMatch = False
    for x in excelSheetName.columns.values:
        xCt += 1
        if xCt == 0:
            continue
        else:
            if xCt < 10 and foundInitialMatch == False: # column idx will not be greater than 9
                m = re.search('named', str(x))
                if m:
                    continue
                else:
                    foundInitialMatch = True
                    xStart = xCt
            else: 
                m = re.search('named', str(x))
                if m:
                    xStop = xCt
                    break
    return (xStart, xStop)
def parsePandasDfRows(col1ListFromPdDf):
    headers = True
    mamuA_indices = []
    mamuB_indices = []
    skipIndices = []
    genotypeList = []
    for idx,val in enumerate(col1ListFromPdDf):
        if headers:
            m = re.search('Comment', str(val))
            mA = re.search('Mamu-A', str(val))
            mB = re.search('Mamu-B', str(val))
            if m:
                headers = False
                skipIndices.append(idx)
                continue
            elif mA:
                mamuA_indices.append(idx)
            elif mB:
                mamuB_indices.append(idx)
            else:
                skipIndices.append(idx)
                continue
        else:
            m = re.search('Alleles', str(val))
            if m:
                skipIndices.append(idx)
            else:
                genotypeList.append(val)
                continue
    return (skipIndices, mamuA_indices, mamuB_indices, genotypeList)
def dataToPandasOneHot(d, pdDf, idxList):
    if pdDf is None:
        pdDf = pd.DataFrame.from_dict(d, orient='index').transpose()
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
    else:
        alleleDFnew = pd.DataFrame.from_dict(d, orient='index').transpose()
        alleleDFnew.index = idxList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, alleleDFnew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    return pdDf
def parseIdxForMamuAMamuB(gList):
    mamuA_nameIdxListCol1 = []
    mamuB_nameIdxListCol1 = []
    for idx, n in enumerate(gList):
        m_MamuA = re.search('Mamu_A', str(n))
        m_MamuB = re.search('Mamu_B', str(n))
        if m_MamuA:
            filterMamu = re.search('Mamu_AG', str(n))
            if filterMamu:
                continue
            else:
                mamuA_nameIdxListCol1.append(idx)
        elif m_MamuB:
            mamuB_nameIdxListCol1.append(idx)
        else:
            continue
    return (mamuA_nameIdxListCol1, mamuB_nameIdxListCol1)

def parseGenotypeList(gList):
    r = []
    for g in gList:
        if type(g) is float:
            r.append(g)
        else:
            gString = g.split('_')
            gStringAsList = gString[1:]
            gStringAsString = '_'.join(gStringAsList)
            r.append(gStringAsString)
    return r

def scanAndUpdateGenotypeList(l1_parsed,l2_training):
    l1_parsed_res = []
    for i in l1_parsed:
        didFindMatch = False
        if (l2_training.count(i) == 1):
            l1_parsed_res.append((1, i))
            continue
        else:
            if type(i) is float:
                l1_parsed_res.append((0, i))
                # see dev comments
                continue
            iString = i.split('g')
            iStringRgx = iString[0]
            for itm in l2_training:
                m1 = re.search(iStringRgx, itm)
                if m1:
                    mString = m1.group(0)
                    if (l2_training.count(mString) == 1):
                        l1_parsed_res.append((2, mString))
                        didFindMatch = True
                        print('WARNING: replacing original item ' + str(i) + ' with modified match from training set: ' + str(mString))
                        break # this is important: it stops at the first match
                else:
                    continue
            if not didFindMatch:
                print('WARNING! Unable to match item ' + str(i))
                print('Item ' + str(i) + ' was deleted from dataset.\nThis behavior will be modified in a future release.')
                l1_parsed_res.append((0, i))
            else:
                continue
    return l1_parsed_res
    
testFile = pd.ExcelFile(pivot_table)
readyToProceed = False
fName_none = ''
dataAsPandas = ''
excelSheetName = parseExcelWithPandas(fName_none, testFile)
processedPivotTable = False
if pivot_table:
    if excelSheetName.empty:
        print('The Excel sheet was found, but there was an error reading the Excel file.')
        print('Please do one of the following:\n\nProceed to the next cell and attempt to enter the sheet_name value,\n\nor\n\nExport the file as a csv, start from the beginning of the python notebook,\nchange the file name, and rerun all cells.\nBe sure to specify the file type as csv in the cell below when you run it.')
    else:
        print('Please click the next cell, and press Run.')
        print('Here is a preview of the data: \n\n##StartPreview:\n')
        pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
        for h in pdHeadersRowsCol1:
            m = re.search('Animal ID', h)
            if m:
                processedPivotTable = True
            print(h + '\n')
        print('##EndPreview\n\nData Type is ')
        if processedPivotTable:
            print('Processed Pivot Table')
        else:
            print('raw pivot table')
        readyToProceed = True
else:
    print('No pivot table was found. Please do the following: \n1) check the filename and rerun the previous cell\n2) if you have already rerun the previous cell and are seeing this message again, \nproceed to the next cell and enter information for at least one of the following: ')
    print('\tfile_type: enter \'csv\' or \'excel\', depending on the file.')
    print('\tsheet_name: enter the sheet name for the excel spreadsheet or csv file')
    print('Then run the next two cells.')

genotypeList_training = openFileAsList('alleles_parsed.txt')

Please click the next cell, and press Run.
Here is a preview of the data: 

##StartPreview:

Animal ID

# Reads Evaluated

# Reads Identified

% Unknown

Mamu-A Haplotype 1

Mamu-A Haplotype 2

Mamu-B Haplotype 1

Mamu-B Haplotype 2

Comments

##EndPreview

Data Type is 
Processed Pivot Table


In [5]:
file_type = '' # must be left blank, 'csv' for a csv file, or 'excel' for an excel file type
sheet_name = '' # must have the excel sheet name with the pivot table, or be blank

Step 2
* This step will reformat the Excel data into a format that can be used by the Machine Learning Classifier.
* Click the next cell, and then click the Run button at the top.
* If you do not see any error messages, and you see the output 'Everything looks good!', then proceed to Step 3.  Otherwise, contact John for assistance with Step 2
  * Note from John: In the Beta build 1.2 this step will be modified to not require intervention from me if something goes wrong.
* If you see a warning message, it is still usually ok to proceed, but you should make a note of the warning.

In [45]:
def parseLabeledValues(l):
    r = []
    for i in l:
        iSplit = i.split('-')
        iSplitSorted = sorted(iSplit)
        iSplitSortdString = '-'.join(iSplitSorted)
        if iSplit[0] == iSplit[1]:
            r.append(iSplit[0])
        else:
            r.append(iSplitSortdString)
    return r

In [47]:
v = findColumnIdxStartStop(excelSheetName)
if v[0] == -1:
    print('Error!  check dataframe!')
col1List = list(excelSheetName['Sample Sheet #'])
skipRows, mamuA_rows, mamuB_rows, genotypeList_unparsed = parsePandasDfRows(col1List)
genotypeList_parsed = parseGenotypeList(genotypeList_unparsed)
genotypeListTuple = scanAndUpdateGenotypeList(genotypeList_parsed, genotypeList_training)
# final step: parse through genotypeListTuple, and remove any rows that correspond to (0, name)
genotypeListFiltered = list(filter(lambda x: x[0] != 0, genotypeListTuple))
genotypeListTupleX, genotypeListTupleY = zip(*genotypeListTuple) # zip iterable, unpacks tuple
genotypeListSkip = [x for (x,y) in enumerate(genotypeListTupleX) if y == 0]
genotypeIntsTuple, genotypeListFromTuple = zip(*genotypeListFiltered) 
genotypeList = list(genotypeListFromTuple)
skipMamuRows = mamuA_rows + mamuB_rows + skipRows + genotypeListSkip
skipMamuRows.sort()
parsedMamuIndices = parseIdxForMamuAMamuB(genotypeList)
parsedMamuAIndices = parsedMamuIndices[0]
parsedMamuBIndices = parsedMamuIndices[1]

# print(genotypeList)
rStart = v[0]
rStop = v[1] + 1 
rRange = rStop - rStart
mamuA_alleleList = []
mamuB_alleleList = []
alleleDF_MamuA = None
alleleDF_MamuB = None
for x in range(rStart, rStop):
    mamu_genotypes_oneHot = []
    dfValue = excelSheetName.iloc[:,x] # verify syntax for rows and columns, and index not lookup
    dfValue_MamuA_1 = dfValue.iloc[int(mamuA_rows[0])]
    dfValue_MamuA_2 = dfValue.iloc[int(mamuA_rows[1])]
    dfValue_MamuB_1 = dfValue.iloc[int(mamuB_rows[0])]
    dfValue_MamuB_2 = dfValue.iloc[int(mamuB_rows[1])]
    pdDict_MamuA = dict()
    pdDict_MamuB = dict()
    for idx, row in dfValue.iteritems():
        if idx not in skipMamuRows:
            try:
                if math.isnan(row):
                    mamu_genotypes_oneHot.append(0)
                    continue
                else:
                    mamu_genotypes_oneHot.append(1)
            except TypeError:
                mamu_genotypes_oneHot.append(1)
    pdDictKey_MamuA = str(dfValue_MamuA_1) + '-' + str(dfValue_MamuA_2)
    pdDictKey_MamuB = str(dfValue_MamuB_1) + '-' + str(dfValue_MamuB_2)
    pdDict_MamuA[pdDictKey_MamuA] = mamu_genotypes_oneHot
    pdDict_MamuB[pdDictKey_MamuB] = mamu_genotypes_oneHot
    alleleDF_MamuA = dataToPandasOneHot(pdDict_MamuA, alleleDF_MamuA, genotypeList)
    alleleDF_MamuB = dataToPandasOneHot(pdDict_MamuB, alleleDF_MamuB, genotypeList)
alleleDF_MamuA_parsed = alleleDF_MamuA.iloc[parsedMamuAIndices,:]
alleleDF_MamuB_parsed = alleleDF_MamuB.iloc[parsedMamuBIndices,:]
alleleDF_MamuA_parsedList = []
alleleDF_MamuA_listLabels_Y = []
for i in range(0, rRange):
    dfColAsSeries = alleleDF_MamuA_parsed.iloc[:,i]
    if dfColAsSeries.name == 'nan-nan':
        continue
    dfColAsList = dfColAsSeries.tolist()
    alleleDF_MamuA_parsedList.append(dfColAsList)
    alleleDF_MamuA_listLabels_Y.append(dfColAsSeries.name)
alleleDF_MamuA_listedLabels_Y = parseLabeledValues(alleleDF_MamuA_listLabels_Y)
alleleDF_MamuA_parsedNP = np.array(alleleDF_MamuA_parsedList)
if not alleleDF_MamuA_parsed.empty and not alleleDF_MamuB_parsed.empty:
    print('Everything looks good!')

Everything looks good!


In [48]:
# remaining steps todo before release of beta 1_0:
# write function to insert results into pivot table
# write logic to handle no haplotype


Step 3
* If you did not encounter any errors previously, or the output did not direct you to stop, then proceed with the analysis.
* You can either click this cell, select the Cell menu above, and then select 'Run All below', or click the cells after this one and individually click 'Run' for each one
* The analysis within this python notebook will do the following:
  * parse out genotype ID's for the MHC-A, and MHC-B data, 
  * use a pre-trained model for a Machine Learning Classifier to predict the Haplotype for each sample for MHC-A and MHC-B
  * add the predicted values back into the pivot table (or create a new pivot table)
  * output the resulting pivot table
* Note that for beta 1.x builds, only MHC-A and MHC-B haplotypes will be predicted.  All other haplotypes will be passed to the researcher for analysis.  A researcher should also verify the output pivot table from the Classifier.

In [49]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.base import clone

In [50]:

def lambdaFunc(v):
    return int(v)

def openAndParse(f):
    listValue = []
    headerLine = ''
    start = True
    with open(f) as fOpen:
        for i in fOpen:
            if start:
                iLine = i.split(',')
                iLine = iLine.pop(0)
                headerLine = ','.join(iLine)
                start = False
            else:
                i = i.rstrip('\n')
                iSplit = i.split(',')
                iSplitInt = list(map(lambdaFunc, iSplit[1:]))
                # listValue.append((iSplit[0], iSplit[1:]))
                listValue.append((iSplit[0], iSplitInt))
    return (headerLine, listValue)

In [51]:
trainingValues = openAndParse('allele_df-trainingSet-HapA.csv')
testingValues = openAndParse('allele_df-testingSet-HapA.csv')

In [52]:
X_train = np.empty([len(trainingValues[1]), len(trainingValues[1][1][1])], dtype=int)
Y_train = np.empty([len(trainingValues[1])], dtype=int)
X_test = np.empty([len(testingValues[1]), len(testingValues[1][1][1])], dtype=int)
Y_test = np.empty([len(testingValues[1])], dtype=int)
trainingValueFromTuple = trainingValues[1]
testingValueFromTuple = testingValues[1]
lookupTableBecauseNumpy = []
lookupTableBecauseNumpyTesting = []
stopRange = len(trainingValueFromTuple)
for x in range(0, stopRange):
    aValue_x = trainingValueFromTuple[x][1]
    aValue_y = trainingValueFromTuple[x][0]
    X_train[x] = aValue_x
    lookupTableBecauseNumpy.append(aValue_y)
    Y_train[x] = np.array(x, dtype=int)
stopRange = len(testingValueFromTuple)
for x in range(0, stopRange):
    aValue_x = testingValueFromTuple[x][1]
    aValue_y = testingValueFromTuple[x][0]
    X_test[x] = aValue_x
    lookupTableBecauseNumpyTesting.append(aValue_y)
    Y_test[x] = np.array(x, dtype=int)

In [ ]:
'''
# Developer Comments JRC
# PRAGMA MARK: Critical Dev comment

A few comments here.
At a basic level, the training set creates a correlation between the one-hot vector of values 
that are fed to it (and was created in this case from the genotypes), with correspinding labeling.  
The labeling is important, otherwise the output would be binary, or a series of integers.  Additionally, labeling can be used in downstream
tasks and more complex machine learning tasks.

There are several algorithms to use for training and prediction, which are too complex to explain here, 
but will be explained and discussed in later developer comments.
The testing set then takes a set of data, which must be formatted as a one-hot vector with corresponding labeling, 
and then predicts how the test dataset will match to the training dataset.  

Initially, testing and training data were from the same pooled data, so creating testing and training data followed similar methods.

To use input data, the steps above--commented out now in the dev version, removed in the beta version--needed to be
modified such that the input data was predicted to have set values, and then those values were matched to a labeling table.

If input data was from a formatted pivot table, then a list of labels are created from the column names.  This becomes Y_test, and accuracy testing can be run.

If the input data was from a raw pivot table, then the classifier makes predictions, and these predictions are mapped back to labeling created from the training data set.  
In this case, no accuracy checks can be done, since these labels are unknown, however, as has been repeatedly stated, the output should be verified by a researcher.

'''

In [60]:
def formatTestingValues(tList, tupleList):
    r = []
    validationInt = len(tList)
    for v in tList:
        for vInTuple in tupleList:
            if v == vInTuple[0]:
                r.append(vInTuple[1])
                break
    if len(r) != len(tList):
        print('WARNING!')
        print(r)
        print(tList)
        return None
    else:
        return r
def valuesToTestList(valList, npArrayTupleListTraining):
    r = []
    matched = False
    for v in valList:
        for vTuple in npArrayTupleListTraining:
            if v == vTuple[1]:
                matched = True
                r.append(vTuple[0])
                break
            # if v == vTuple[0]:
                # matched = True
                # r.append(vTuple[1])
                # break
        if matched:
            matched = False
            continue
        else:
            print('WARNING!')
            print(v)
            print(npArrayTupleListTraining)
            return None
    return r
        
    
def valuesToIntList(valList):
    listedValues = []
    ct = 0
    l_strings = []
    l_ints = []
    l_strings_r = []
    l_ints_r = []
    npArrayList = []
    for i in valList:
        if i not in l_strings:
            l_strings_r.append(i)
            l_ints_r.append(ct)
            l_strings.append(i)
            l_ints.append(ct)
            npArrayList.append(ct)
            ct += 1
        else:
            v_i = l_strings.index(i)
            v_i_string = l_strings[v_i]
            v_ct = l_ints[v_i]
            l_strings_r.append(v_i_string)
            l_ints_r.append(v_ct)
            npArrayList.append(v_ct)
    npArrayTupleList = list(zip(l_strings_r,l_ints_r))
    return (npArrayList, npArrayTupleList)

parsedTrainingResults = valuesToIntList(lookupTableBecauseNumpy)
Y_train = np.array(parsedTrainingResults[0])
parsedTestingResults = valuesToIntList(lookupTableBecauseNumpyTesting)
Y_testList = valuesToTestList(alleleDF_MamuA_listedLabels_Y, parsedTrainingResults[1])
Y_test = np.array(Y_testList)
# print(Y_testList)

WARNING!
A008-A025
[('A004', 0), ('A004', 0), ('A004', 0), ('A004', 0), ('A004-A025', 1), ('A002a-A004', 2), ('A002a-A004', 2), ('A004-A011', 3), ('A004-A023', 4), ('A001-A008', 5), ('A004-A023', 4), ('A002a-A006', 6), ('A004-A025', 1), ('A004', 0), ('A002a-A004', 2), ('A004-A023', 4), ('A004-A008', 7), ('A001', 8), ('A001', 8), ('A001-A004', 9), ('A004-A011', 3), ('A004-A008', 7), ('A004-A008', 7), ('A004', 0), ('A004', 0), ('A004', 0), ('A004-A023', 4), ('A001-A002a', 10), ('A004', 0), ('A001-A002a', 10), ('A004', 0), ('A002a-A008', 11), ('A004', 0), ('A001', 8), ('A001-A008', 5), ('A001-A008', 5), ('A004-A011', 3), ('A002a-A006', 6), ('A004-A008', 7), ('A001-A008', 5), ('A004-A008', 7), ('A002a-A008', 11), ('A004-A008', 7), ('A004', 0), ('A004-A023', 4), ('A004-A023', 4), ('A001-A004', 9), ('A002a-A004', 2), ('A004', 0), ('A001-A002a', 10), ('A004-A008', 7), ('A004-A008', 7), ('A004', 0), ('A002a-A004', 2), ('A001-A004', 9), ('A004', 0), ('A001-A004', 9), ('A002a-A004', 2), ('A004',

In [ ]:
if processedPivotTable:
    if processedPivotTableList:
        print() # run function
    else:
        print() # # throw warning, run prediction function
else:
    print() # run prediction function

In [54]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = [{'weights': ["uniform", "distance"]}]

# KNearestNeighbors was used, but a different library may be applied later
knn_clf = KNeighborsClassifier(n_jobs=-1, weights='distance', n_neighbors=4)
knn_clf.fit(X_train, Y_train)

y_knn_pred = knn_clf.predict(X_test)

# forest_clf_pred = forest_clf.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_knn_pred)
# This accuracy score strictly tests the model, and may be subject to overfitting

TypeError: Singleton array array(None, dtype=object) cannot be considered a valid collection.

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, Y_train)
forest_clf_pred = forest_clf.predict(X_test)

accuracy_score(Y_test, forest_clf_pred)
print(forest_clf_pred)
print(Y_test)




[10  8 10  9  9 10  0  0  7  4  0  0  4  5  7  7  6  2  0  2  0]
[10  8 10  9  9 10  0  0  7  4  0  0  4  5  7  7  6  2  0  2  0]


In [58]:
sgd_clf = SGDClassifier(random_state=42)
cross_val_score(sgd_clf, X_train, Y_train, cv=2, scoring="accuracy")

/Users/thor/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/thor/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([ 0.84782609,  0.92682927])